In [5]:
import pyodbc 
import os

sql_conn_str = os.getenv('MSSQL_CONN_STRING_DOCKER') or ''
cnxn = pyodbc.connect(sql_conn_str)

InterfaceError: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [3]:
def find_children(node, rows):
    appending=True
    children = []
    ids = set([node[0]])
    while appending:
        length = len(children)
        children_nodes = [r for r in rows if r[1] in ids]        
        children.extend(children_nodes)
        children = list(set([ tuple(p) for p in children]))
        appending = (length != len(children))
    return children


def build_main_equipment_reverse_lookup():
    sql = """
      SELECT 
          mq.ID, mq.Parent_ID, mq.isMainEquipment, mo.ID as Model_ID, mq.TechnicalInfoTag_ID
      FROM 
          ModelHierarchy mq left join model mo on mo.ModelHierarchy_ID = mq.ID
       """
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    rows = cursor.fetchall() 

    print(f"Total number of models in IOC: {len(rows)}")
    rows = [list(x) for x in rows]
    rows_main_equipment = [r for r in rows if r[2]]
    print(f"Main equipment models in IOC: {len(rows_main_equipment)}")

    reverse_results = {}
    for parent in rows_main_equipment:
        # Reverse lookup dictionary
        for x in find_children(parent, rows):
            reverse_results[x[3]]=parent[0] 
    print(f"Models with a parent main equipment: {len(reverse_results)}")
    return reverse_results


def find_stid_tag(model_hierarchy_id):
    sql = f"""
        SELECT t.*
        FROM TechnicalInfoTag t, ModelHierarchy m
        WHERE m.id = '{model_hierarchy_id}' 
            and t.id = m.TechnicalInfoTag_ID
        """
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    rows = cursor.fetchall() 
    if not rows:
        raise Exception(f"Cannot find stid tag for Model hierarchy:{model_hierarchy_id}")
    else:
        return rows[0]

    
def find_stid_children(stid_id):
    # This takes 16'' too slow, 3M table
    sql=f"""
        WITH tit_children AS (
            SELECT       
                ID, 
                Tag,
                Parent_ID
            FROM       
                TechnicalInfoTag
            WHERE 
                ID = '{stid_id}'
            UNION ALL
            SELECT 
                ti.ID, 
                ti.Tag,
                ti.Parent_ID
            FROM 
                TechnicalInfoTag ti
                INNER JOIN tit_children tic 
                    ON tic.ID = ti.Parent_ID
        )
        SELECT *
        FROM 
            tit_children tic
    """
    
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    rows = cursor.fetchall()
    return rows


def find_notifications(stid_tags):
    str_stid_tags = "','".join(stid_tags)
    sql = f"""
        SELECT * 
        FROM  Notification n
        WHERE n.TechnicalInfoTag_ID in ('{str_stid_tags}')
    """
    cursor = cnxn.cursor()
    cursor.execute(sql) 
    
    columns = [column[0] for column in cursor.description]
    results = []
    for row in cursor.fetchall():
        results.append(dict(zip(columns, row)))
    return results
    
    
    
reverse_me_lookup = build_main_equipment_reverse_lookup()

NameError: name 'cnxn' is not defined

In [4]:
from pprint import pprint

cnxn = pyodbc.connect(sql_conn_str)
model_id  = ""
try:
    main_equipment_parent_hierarchy_id = reverse_me_lookup[model_id]
except KeyError:
    print(f"Can't find Main equipment for model {model_id}")

stid_tag = find_stid_tag(main_equipment_parent_hierarchy_id)

stid_children = find_stid_children(stid_tag[0])




Stid ID: 43A078C8-0524-4707-83B0-BAAB3249E19B
Rows: [('43A078C8-0524-4707-83B0-BAAB3249E19B', '23KA0003', 'FA821F42-3908-49F8-8EB3-453C12828C64'), ('E0959123-17FC-4869-A50D-00474102D8AC', '23PDT0412', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('A88B6C8E-E885-44B6-9563-0466DD33ECBA', '23PST0351', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('A86FE44D-7DB9-494C-B5CD-068ADAF2572D', '23PT2028', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('DCB35294-B689-4DD8-8023-0C2E80A7CBE4', '23ZT2049B', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('812F4A0B-2E6D-42BF-AA62-0CD6BB5B2010', '23PST0452', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('84490615-275E-498E-A7FA-0D53693B60DC', '23TT0350', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('560E17AE-20F1-431C-8888-0D9902E70889', '23TT2040', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('74F139E6-1086-4AAA-B967-0DD1D13881C9', '23PSV0456A', '43A078C8-0524-4707-83B0-BAAB3249E19B'), ('2AE92B23-9FF5-4E2A-8BCF-11D20D829E50', '23FT2026', '43A078C8-0524-4707-83B0-BAAB3249E19B'), (

In [7]:
notifications = find_notifications([x[0] for x in stid_children])
print(f"Found:{len(notifications)}")
pprint(notifications)


Found:14
[{'BreakdownIndicator': 'False',
  'Created': datetime.datetime(2019, 8, 30, 1, 19, 56, 407000),
  'CreationDate': datetime.datetime(2019, 8, 14, 8, 7, 12),
  'CreationDateUTC': datetime.datetime(2019, 8, 14, 6, 7, 12),
  'CreationTime': datetime.time(8, 7, 12),
  'EquipmentNumber': '11779901',
  'FailureImpact': 'X',
  'FailureImpactText': 'No Failure - Other needs',
  'FailureMechanism': '    ',
  'FailureMechanismText': '',
  'FailureMode': 'OTH ',
  'FailureModeText': 'ISO Pumps - CST Other',
  'ID': '570524A2-4BD6-4EC1-9A25-0D9481795A0A',
  'LastChangeDate': datetime.datetime(2019, 9, 28, 1, 5, 19),
  'LastChangeDateUTC': datetime.datetime(2019, 9, 27, 23, 5, 19),
  'LongText': 'Beskrivelse av redusert funksjon eller feiltilstand: Vi får '
              'fremdeles høy temperatur på lageret NDE ved smøring. Smøringen '
              'ble justert 15.07.19 på AO 24728091. Vi har vært ute og kjølt '
              'med luft flere ganger den siste uka. ************* Trend lengr